## 2. Wright (1928)

#### 1. Consider the canonical demand and supply model in which quantity supplied is a function of price and a set of “supply shifters”; quantity demanded is a function of price and set of “demand shifters”; and market clearing implies that at some price quantity demanded is equal to quantity supplied.
#### Consider the following questions:
#### (1) **(Control)** What is the expected demand if we ***set*** the price $p = p_0$?

Let 
$$ q_D = D(p, u) $$
$$ q_S = S(p, v) $$
where $u$ and $v$ are exogeneous shocks that affect demand and supply, respectively.

Note that $p$, $q_D$, and $q_S$ are endogeneous.

If we ***set*** the price $p = p_0$,
* The price is no longer random;
* Since $u$ is exogeneous to the model (external factor), setting the price (internal mechanism) does not affect the distribution of $u$.
* Note that $u$ and $v$ are still random since neither is affected by setting the price.

The expected demand if we ***set*** the price $p = p_0$ is:
$$ \mathbb{E}(q_D(p_0)) = \int q_D(p_0, u) dF_u(u) $$

#### (2) **(Condition)** What is the expected demand if we ***observe*** the price $p = p_0$?

If we ***observe*** the price $p = p_0$,
* The observed price is an equilibrium outcome of supply and demand, which are both influenced by $u$ and $v$.
* In other words, the exogeneous shocks $u$ and $v$ influence the price $p$ and the quantity $q$ through demand and supply curves.
* If we observe a higher price (and assuming $q_D$ and $q_S$ are monotonically increasing in $u$ and $v$, respectively,
    * Since a higher demand for any given supply level would increase prices, the exogeneous shock that affects demand, $u$, is likely to be higher.
    * Since a lower supply for any given demand level would increase prices, the exogeneous shock taht affects supply, $v$, is likely to be lower.
 
The expected demand if we ***observe*** the price $p = p_0$ is:
$$ \mathbb{E}(q^*|p=p_0) = \mathbb{E}[q^*(u,v)|q_D(p_0,u) =  q_S(p_0, v)] $$

#### (2) **(Counterfactural)** What **would** demand be if we **were** to ***change*** the price to $p_1$, *ceteris paribus*?

Let $(p_0, q_0)$ be initial equilibrium price and quantity outcomes due to $u_0$ and $v_0$ shocks.
Note that $u_0$ and $v_0$ are no longer random here as they are specific values that correspond to the observed equilibrium.

If we **were** to ***change*** (say, increase WLOG), the price to $p_1$, *ceteris paribus*,
* The quantity demand will decrease, holding other factors fixed.
* That is, since $u$ and $v$ are no longer random here, we're only moving along the demand curve for a change in price.

The change in demand is:
$$ \triangle = q_D(p_1, u_0) - q_D(p_0,u_0) $$
which is negative for $p_1 > p_0$.